In [6]:
import numpy as np
import torch
import torch.nn as nn

X=np.ones((6,8))
X[:,2:6]=0
X


array([[1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.]])

In [17]:
Y = np.zeros((6,7))
Y[:,1]=1
Y[:,5]=-1
Y

array([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [22]:
# 构造一个二维卷积层，它具有1个输出通道和形状为（1，2）的卷积核
conv2d = nn.Conv2d(1,1, kernel_size=(1, 2), bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度），
# 其中批量大小和通道数都为1
X=torch.tensor(X,dtype=torch.double)
Y=torch.tensor(Y,dtype=torch.double)
X = X.reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))
lr = 3e-2  # 学习率

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核
    conv2d.weight.data[:] -= lr * conv2d.weight.grad
    if (i + 1) % 2 == 0:
        print(f'epoch {i+1}, loss {l.sum():.3f}')

C:\Users\Liu Xinyu\AppData\Local\Temp\ipykernel_26888\3561829020.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X=torch.tensor(X,dtype=torch.double)
C:\Users\Liu Xinyu\AppData\Local\Temp\ipykernel_26888\3561829020.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y=torch.tensor(Y,dtype=torch.double)


RuntimeError: expected scalar type Double but found Float

In [2]:



class MyNet1(nn.Module):
    def __init__(self, linear1, linear2):
        super(MyNet1, self).__init__()
        self.linear1 = linear1
        self.linear2 = linear2

    def forward(self, X):
        return self.linear2(self.linear1(nn.Flatten()(X)))


class MyNet2(nn.Module):
    def __init__(self, linear, conv2d):
        super(MyNet2, self).__init__()
        self.linear = linear
        self.conv2d = conv2d

    def forward(self, X):
        X = self.linear(nn.Flatten()(X))
        X = X.reshape(X.shape[0], -1, 1, 1)
        X = nn.Flatten()(self.conv2d(X))
        return X


linear1 = nn.Linear(15, 10)
linear2 = nn.Linear(10, 5)
conv2d = nn.Conv2d(10, 5, 1)
linear2.weight = nn.Parameter(conv2d.weight.reshape(linear2.weight.shape))
linear2.bias = nn.Parameter(conv2d.bias)
net1 = MyNet1(linear1, linear2)
net2 = MyNet2(linear1, conv2d)
X = torch.randn(2, 3, 5)
# 两个结果实际存在一定的误差，直接print(net1(X) == net2(X))得到的结果不全是True
print(net1(X))
print(net2(X))
print(net1(X)==net2(X))

tensor([[-0.0306, -0.1093,  0.4894, -0.2966,  0.0861],
        [-0.0837, -0.2906, -0.4746, -0.0287, -0.0200]],
       grad_fn=<AddmmBackward0>)
tensor([[-0.0306, -0.1093,  0.4894, -0.2966,  0.0861],
        [-0.0837, -0.2906, -0.4746, -0.0287, -0.0200]],
       grad_fn=<ViewBackward0>)
tensor([[False, False, False, False,  True],
        [False,  True,  True, False, False]])


In [71]:
c = torch.nn.Conv2d(1, 1, (2, 2), stride=2, padding=1, bias=False)
c.weight.data = torch.Tensor([[[[1, -1],
                                [-1, 1]]]])
c.weight.data.dim()

4

In [68]:
a = torch.Tensor([[[[1, 0, 0, 0, 2],
                    [0, 1, 0, 0, 0],
                    [0, 0, 1, 0, 0],
                    [0, 0, 0, 2, 2],
                    [0, 0, 0, 0, 0]]]])
print(c(a))
a.dim()

tensor([[[[1., 0., 2.],
          [0., 2., 0.],
          [0., 0., 0.]]]], grad_fn=<ConvolutionBackward0>)


4

In [61]:
net = nn.Conv1d(1,1, kernel_size=2, padding=1)
net.weight.data = Y
net.weight

Parameter containing:
tensor([[[ 1., -1.],
         [-1.,  1.]]], requires_grad=True)

In [62]:
net(X)

RuntimeError: Given groups=1, weight of size [1, 2, 2], expected input[1, 3, 4] to have 2 channels, but got 3 channels instead